## 完整预测流程
### 1.导入必要的包

In [1]:
from email import header
import json
import numpy as np
import pandas as pd

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense
from tools.Attention import Attention
from keras.layers import GRU, Bidirectional
from tqdm import tqdm
from tkinter import _flatten

from pandarallel import pandarallel 
from keras.models import load_model
import config as cfg
import sys
sys.path.append("../tools/")
import tools.funclib as funclib

pandarallel.initialize() 

2022-11-07 09:10:04.577372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 09:10:04.672483: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:

# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280

embd_feature = pd.read_feather(f"/home/dengrui/DMLF/data/featureBank/embd_{EMBD_METHO}.feather")

ecs_2 = [True, False]
ecs_8 = [1,2,3,4,5,6,7,8]
label_multi = pd.read_feather('/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/label_multi.feather')
ecs_multi = label_multi['label_multi'].tolist()

In [3]:
# test_task1_2022 = pd.read_feather('/home/dengrui/DMLF/data/datasets/task1/test_2022.feather')
# test_task2_2022 = pd.read_feather('/home/dengrui/DMLF/data/datasets/task2/test_2022.feather')
# test_task3_2022 = pd.read_feather('/home/dengrui/DMLF/data/datasets/task3/test_2022.feather')



# test_set_task1= test_task1_2022.merge(embd_feature, on='id', how='left')
# test_set_task2= test_task2_2022.merge(embd_feature, on='id', how='left')
# test_set_task3= test_task3_2022.merge(embd_feature, on='id', how='left')

# test_task1_feature = test_set_task1.iloc[:, np.r_[3:EMBD_WH+3]]
# x_test_task1 = np.array(test_task1_feature)
# x_test_task1 =np.reshape(x_test_task1,(x_test_task1.shape[0],1,EMBD_WH))


# test_task2_feature = test_set_task2.iloc[:, np.r_[3:EMBD_WH+3]]
# x_test_task2 = np.array(test_task2_feature)
# x_test_task2 =np.reshape(x_test_task2,(x_test_task2.shape[0],1,EMBD_WH))

# test_task3_feature = test_set_task3.iloc[:, np.r_[3:EMBD_WH+3]]
# x_test_task3 = np.array(test_task3_feature)
# x_test_task3 =np.reshape(x_test_task3,(x_test_task3.shape[0],1,EMBD_WH))



## 读取测试数据

In [4]:
# 读取测试数据
test = pd.read_csv('/home/dengrui/DMLF/data/corynebacterium.tsv',sep='\t')[['id','seq','ec']]


# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280

data = pd.read_feather(f"/home/dengrui/DMLF/data/featureBank/embd_{EMBD_METHO}.feather")
test= test.merge(data, on='id', how='left')
test_feature = test.iloc[:, 3:]
test_feature = np.array(test_feature)
test_feature =np.reshape(test_feature,(test_feature.shape[0],1,EMBD_WH))

# 测试结果
test_res= test[['id','seq', 'ec']].copy()


# task1

In [5]:
from keras.models import load_model
listec = list()
load_model = load_model("/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task1_2022.h5",custom_objects={"Attention": Attention},compile=False)
predicted = load_model.predict(test_feature)
test_res['pred_isenzyme'] = pd.DataFrame(np.argmax(predicted, axis=-1)).astype(bool)

2022-11-07 09:10:19.493657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 09:10:20.084606: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46702 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:86:00.0, compute capability: 8.6


 1/97 [..............................] - ETA: 2:49

2022-11-07 09:10:23.277080: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-11-07 09:10:23.427320: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


97/97 [==============================] - 2s 1ms/step


# taks2

In [6]:
from keras.models import load_model

listec = list()
load_model = load_model("/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task2_model_2022_label8.h5",custom_objects={"Attention": Attention},compile=False)
predicted = load_model.predict(test_feature)
counter = 0
for i in tqdm(range(len(test))):
    listec.append(sorted(dict(zip(set(ecs_8),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])

test_res['pred_functions']=listec

97/97 [==============================] - 1s 1ms/step


100%|██████████| 3098/3098 [00:00<00:00, 345961.12it/s]


# task3

In [7]:
def make_rec_str(pred_top):
    pred = [item[0]+'('+str(item[1])+')' for item  in pred_top]
    pred = ', '.join(pred)
    return pred

def make_multi_ec_pred(pred_func, pred_top_list):
    res_ecs = pred_top_list[:pred_func]
    res_ecs = [item[0] for item in res_ecs]
    return ','.join(res_ecs)

from keras.models import load_model
listec = list()
listpred = list()
model = load_model(f'/home/dengrui/anaconda3/PyProjects/MutiLabelProjects/multi-label-classification-4-event-type/multi-label-classification-4-event-type/task3_{EMBD_METHO}_2022.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model.predict(test_feature)
counter = 0
for i in tqdm(range(len(test_feature))):
    listpred.append(sorted(dict(zip((ecs_multi),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])
    listend= list()
    sortlist = sorted(dict(zip((ecs_multi),predicted[i])).items(),key = lambda x:x[1], reverse = True)
    # listpred.append(sortlist[0])
    for j in range(0,len(sortlist)):
        if j == 20:
            break
        listend.append([sortlist[j][0], round(sortlist[j][1],4)])
    listec.append(listend)

test_res['pred_ec']=listpred
test_res['pred_top20'] = listec
test_res['pred_ec'] = test_res.apply(lambda x : x.pred_ec if x.pred_functions <=1 else make_multi_ec_pred(pred_func=x.pred_functions, pred_top_list=x.pred_top20), axis=1)
test_res['pred_top20'] = test_res['pred_top20'].apply(lambda x : make_rec_str(pred_top=x))
test_res['pred_ec'] = test_res.apply(lambda x : x.pred_ec if x.pred_isenzyme ==True else '-', axis=1)
test_res

97/97 [==============================] - 1s 1ms/step


100%|██████████| 3098/3098 [00:13<00:00, 223.66it/s]


,id,seq,ec,pred_isenzyme,pred_functions,pred_ec,pred_top20
0,Q79VD7,MTAVAPRVDGHVAPQRPEPTGHARKGSKAWLMMTTTDHKQLGIMYI...,7.1.1.9,True,1,1.9.3.1,"1.9.3.1(0.9978), 1.10.3.-(0.0359), 1.10.3.10(0..."
1,Q8NRC3,MSSASTFGQNAWLVDEMFQQFQKDPKSVDKEWRELFEAQGGPNTTP...,1.2.4.2; 2.3.1.61,True,4,"1.2.4.2,2.3.1.61,2.2.1.5,4.1.1.71","1.2.4.2(0.9986), 2.3.1.61(0.9962), 2.2.1.5(0.9..."
2,Q8NN33,MNSPQNVSTKKVTVTGAAGQISYSLLWRIANGEVFGTDTPVELKLL...,1.1.1.37,True,1,1.1.1.37,"1.1.1.37(0.9992), 1.1.1.82(0.0014), 1.1.1.299(..."
3,Q8NNJ2,MAFSVEMPELGESVTEGTITQWLKSVGDTVEVDEPLLEVSTDKVDT...,2.3.1.12,True,1,2.3.1.12,"2.3.1.12(0.9444), 2.3.1.61(0.0781), 6.4.1.3(0...."
4,P04964,MTNIRVAIVGYGNLGRSVEKLIAKQPDMDLVGIFSRRATLDTKTPV...,1.4.1.16,True,1,1.4.1.16,"1.4.1.16(0.9176), 1.4.1.21(0.0804), 1.2.1.10(0..."
...,...,...,...,...,...,...,...
3093,Q8NUD0,MTSLRDLLVSTNADAAIADMSAFIEETVGKQSGISGIALKGAMGAA...,NaN,False,1,-,"3.-.-.-(0.0167), 2.3.2.-(0.0074), 2.3.1.-(0.00..."
3094,Q8NUD1,MSNPNKKPLDPAQRFSLPTRWAMFFVGIFIMSFAIGITVHAGLGTT...,NaN,False,1,-,"2.3.1.-(0.0111), 2.3.2.3(0.0043), 2.3.1.15(0.0..."
3095,Q8NUD2,MIQTYILKIQSLKWAQREMLKTTLITDTLGHVTSTEPTTTAHVPLP...,NaN,True,1,3.1.1.-,"3.1.1.-(0.2296), 2.3.1.-(0.1521), 3.1.1.3(0.03..."
3096,Q8NUD7,MIVDNFVYLWIDFKVRILSPVCADVSPAYPQFLGSCAQGGKFRGIT...,NaN,False,1,-,"3.1.4.35(0.0723), 3.4.21.-(0.0305), 3.2.1.35(0..."


## balst

In [8]:
sprot = pd.read_feather(cfg.DATADIR+'uniprot/snap201802.feather')
test.ec = test.ec.fillna('-')
ec_sprot = _flatten([item.split(',') for item in list(set(sprot.ec_number))])


res_diamond = funclib.getblast(train=sprot[['id','seq']], test=test[['id','seq']])
res_diamond = res_diamond [['id','sseqid','pident']].merge(sprot[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_diamond = res_diamond[['id_x', 'ec_number','pident']].rename(columns={'id_x':'id','ec_number':'ec_diamond'})
test_res = test_res.merge(res_diamond, on='id', how='left').drop_duplicates().reset_index(drop=True)
test_res.ec_diamond = test_res.ec_diamond.fillna('-')

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [12]:
test_res[['id', 'pred_ec', 'pred_top20']].rename(columns={'pred_ec':'ec_ours'}).to_csv(cfg.RESULTSDIR+'case_gubang/dmlf/corynebacterium_pred.tsv', index=None, sep='\t')